In [ ]:
%matplotlib inline
import os
import re
import string
import shutil

from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import scipy as sp

import sklearn as sk
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight

from gensim.models import KeyedVectors

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, LSTM, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding, Flatten, Concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tqdm import tqdm, tqdm_notebook, tnrange, trange

In [ ]:
tqdm.monitor_interval = 0
tqdm_notebook().pandas()

In [ ]:
# all_comments = pd.concat([train.cleaned_comments,test.cleaned_comments])
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(all_comments)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))
# train_sequences = tokenizer.texts_to_sequences(train.cleaned_comments)
# test_sequences = tokenizer.texts_to_sequences(test.cleaned_comments)
# sequence_pad_len=5000
# train_data = pad_sequences(train_sequences, maxlen=sequence_pad_len)
# test_data = pad_sequences(test_sequences, maxlen=sequence_pad_len)

In [ ]:
# max_comment_len = 0
# big_lim = 4000
# big_lens = {'set_name':list(),'index':list(),'comment_len':list()}
# i=0
# for s in train_sequences:
#     max_comment_len=max(max_comment_len, len(s))
#     if len(s) > big_lim:
#         big_lens['set_name'].append('train')
#         big_lens['index'].append(i)
#         big_lens['comment_len'].append(len(s))
#     i += 1
# i=0
# for s in test_sequences:
#     max_comment_len=max(max_comment_len, len(s))
#     if len(s) > big_lim:
#         big_lens['set_name'].append('test')
#         big_lens['index'].append(i)
#         big_lens['comment_len'].append(len(s))
#     i += 1
# big_lens_df = pd.DataFrame(big_lens)
# max_comment_len
# import seaborn as sn
# sn.distplot(a=big_lens_df.comment_len)

In [ ]:
# embedding_dim = 25
# word2vec = KeyedVectors.load_word2vec_format(
#             os.path.join("W2V",
#                 'w2v.twitter.27B.%dd.txt' % embedding_dim),
#             binary=False)

In [ ]:
# embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
# for word, i in word_index.items():
#     if word in word2vec.wv.vocab:
#         embedding_matrix[i] = word2vec.wv[word]

In [ ]:
# keras.backend.clear_session()
# conv_model = Sequential()
# conv_model.add(Embedding(len(word_index) + 1, 
#                             embedding_dim, 
#                             weights=[embedding_matrix], 
#                             input_length=sequence_pad_len)
#               )
# conv_model.add(Conv1D(128, 5, activation='relu'))
# conv_model.add(MaxPooling1D(5))
# conv_model.add(Conv1D(128, 5, activation='relu'))
# conv_model.add(MaxPooling1D(5))
# conv_model.add(Conv1D(128, 5, activation='relu'))
# conv_model.add(MaxPooling1D(35))
# conv_model.add(Flatten())
# conv_model.add(Dense(128, activation='relu'))
# conv_model.add(Dense(128, activation='relu'))
# conv_model.add(Dense(len(cats), activation='hard_sigmoid'))

# model.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['acc', custom_metric])
# model.summary()

In [ ]:
# model.fit(x=train_data, y=Y_train, batch_size=32, sample_weight=samp_weight,epochs=2)

In [ ]:
# test_model(model, train_data)

In [ ]:
sample_weights = compute_sample_weight('balanced', Y_train)

In [ ]:
keras.backend.clear_session()
#input layer
MLP_model = Sequential()
MLP_model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
#hidden layers
MLP_model.add(Dense(256, activation='relu'))
MLP_model.add(Dense(256, activation='relu'))
MLP_model.add(Dense(256, activation='relu'))
#output layer
# MLP_model.add(Dense(len(cats), activation='hard_sigmoid'))
MLP_model.add(Dense(1, activation='hard_sigmoid'))

MLP_model.compile(optimizer='adam',
             loss='mse',
             metrics=['acc', custom_metric])
MLP_model.summary()

In [ ]:
MLP_model.fit(x=X_train, y=Y_train[cat], batch_size=256, sample_weight=sample_weights, epochs=3)

In [ ]:
# MLP_model.optimizer.lr = keras.backend.variable(1e-4, name='lr')
MLP_model.fit(x=X_train, y=Y_train[cat], batch_size=256, sample_weight=sample_weights, epochs=5)

In [ ]:
MLP_model.fit(x=X_train, y=Y_train[cat], batch_size=256, sample_weight=sample_weights, epochs=10)

In [ ]:
MLP_model.fit(x=X_train, y=Y_train[cat], batch_size=256, epochs=3)

In [ ]:
test_model(MLP_model, X_train, cat)

In [ ]:
test_input = selector.transform(cvect.transform(test.cleaned_comments))

In [ ]:

test_preds = MLP_model.predict(test_input , batch_size=256,verbose=1)

In [ ]:
import pickle 
f = open('%s_preds.P' % cat,'wb')
pickle.dump(test_preds, f)
f.close()

In [ ]:
MLP_model.save('%s.keras.mdl' % cat)

In [ ]:
train.where(train['severe_toxic'] == True).where(train['toxic']  != True).dropna(how='all')